# `logictools` WaveDrom 튜토리얼
[WaveDrom](http://wavedrom.com) 은 타이밍 파형 랜더링을 위한 도구이다. 파형은 간단한 텍스트 포멧으로 정의된다. 이번 챕터는 PYNQ 라이브러리로 어떻게 디지털 파형을 랜더링하는지 알려줄 것이다.

__`logictools`__ overlay는 보드에서 실제 신호를 만드는 WaveDrim과 유사한 포멧을 사용한다.

WaveDrom에 대한 전체 튜토리얼은 여기에서 찾을 수 있다. [here](http://wavedrom.com/tutorial.html)

### Step 1: PYNQ 라이브러리에서 `draw_wavedrom()` 메소드를 가져온다.

In [2]:
from pynq.lib.logictools.waveform import draw_wavedrom

Jupyter notebook 안에 wavedrom diagram을 추가하는 간단한 함수가 있다. 이것은 wavedrom 자바 스크립트 라이브러리를 활용한다.

<font color="DodgerBlue">**Example usage:**</font> 
```python
        from pynq.lib.logictools.waveform import draw_wavedrom   
        
        clock = {'signal': [{'name': 'clk', 'wave': 'h....l...'}]}   
        draw_wavedrom(clock)   
```       
<font color="DodgerBlue">**Method:**</font> 
```python
        def draw_wavedrom(data, width=None):            
        # Note the optional argument width forces the width in pixels
```

### Step 2: 파형을 지정하고 랜더링한다.

In [3]:
from pynq.lib.logictools.waveform import draw_wavedrom

clock = {'signal': [{'name': 'clock_0', 'wave': 'hlhlhlhlhlhlhlhl'}],
         'foot': {'tock': 1},
         'head': {'text': 'Clock Signal'}}

draw_wavedrom(clock)

__Notes on waveform specification__

![](./images/waveform_spec_format0.png)

### Step 3: 파형에 더 많은 신호들을 가한다.

In [4]:
from pynq.lib.logictools.waveform import draw_wavedrom

pattern = {'signal': [{'name': 'clk', 'wave': 'hl' * 8},
                      {'name': 'clkn', 'wave': 'lh' * 8},
                      {'name': 'data0', 'wave': 'l.......h.......'},
                      {'name': 'data1', 'wave': 'h.l...h...l.....'}],
           'foot': {'tock': 1},
           'head': {'text': 'Pattern'}}

draw_wavedrom(pattern)

__Notes on waveform specification__

![](./images/waveform_spec_format2.png)

__여러 파형 그룹과 공백들을 추가한다.__

In [5]:
from pynq.lib.logictools.waveform import draw_wavedrom

pattern_group = {'signal': [['Group1',
                             {'name': 'clk', 'wave': 'hl' * 8},
                             {'name': 'clkn', 'wave': 'lh' * 8},
                             {'name': 'data0', 'wave': 'l.......h.......'},
                             {'name': 'data1', 'wave': 'h.l...h...l.....'}],
                            {},
                            ['Group2',
                             {'name': 'data2', 'wave': 'l...h..l.h......'},
                             {'name': 'data3', 'wave': 'l.h.' * 4}]],
                 'foot': {'tock': 1},
                 'head': {'text': 'Pattern'}}

draw_wavedrom(pattern_group)

__Notes on waveform specification__

![](./images/waveform_spec_format2.png)

# 실시간 패턴 생성 및 추적 분석을위한 WaveDrom

###  __`logictools`__ overlay 는 보드에서의 실제 신호를 지정하고 검출하는데 쓰이는 WaveJSON 포맷을 사용한다.

![](./images/logictools_block_diagram.png)

* 위의 그림에서 볼 수 있듯이, 패턴 생성기는 아두이노 인터페이스의 출력 핀에 나타나는 일련의 논리 값 (패턴)을 지정하는 출력 전용 블록이다. 패턴생성기용 logictools API는 ** WaveDrom ** Secification Syntax에 대해 일부 개선되었다.

   
* Trace Analyzer는 모든 IO 신호를 캡처하고 기록하는 입력 전용 블록이다. 이 신호들은 외부 회로에 의해 구동되는 PL에 대한 입력이거나 생성기에 의해 구동되는 출력 일 수있다. 추적 분석기를 사용하면 생성기에서 지정한 출력 신호가 올바르게 적용되고 있는지 확인할 수 있다. 또한 외부 인터페이스의 작동을 디버깅하고 분석 할 수 있다.
   
   
* 두 블록에서 생성되거나 캡처 된 신호는 이 notebook에서 우리가 본 WaveJSON dictionary를 채워 노트북에 표시 할 수 있다. 사용자는 제공된 API를 통해 사전에 액세스하여 특수 주석으로 파형을 확장하거나 수정할 수 있다.


* 우리는 패턴 생성기의 파형을 지정하기 위해  WaveDrom에서 지원하는 wave tokens의 하위 집합을 사용한다. 그러나 유저들은 주석을 달아 dictionary을 확장 및 수정하기 위해 dictionary에서`draw_waveform ()`메소드를 호출할 수 있다.


__아래의 예에서는 패턴 생성기를 사용하여 Arduino 인터페이스 핀 D0, D1 및 D2에서 3 개의 신호를 생성한다. 모든 I/O는 Trace Analyzer에서 액세스 할 수 있으므로 핀들의 데이터도 캡처할 수 있다. 이 작업은 내부 루프백의 역할을 한다.__

### Step 1: `Logictools` Overlay를  다운로드 받고 패턴을 지정한다.

패턴은 WaveJSON 형식으로 지정되어 생성된다. Waveform 클래스는 지정된 파형을 표시하는 데 사용된다.

In [6]:
from pynq.lib.logictools import Waveform
from pynq.overlays.logictools import LogicToolsOverlay
from pynq.lib.logictools import PatternGenerator

logictools_olay = LogicToolsOverlay('logictools.bit')

loopback_test = {'signal': [
    ['stimulus',
     {'name': 'output0', 'pin': 'D0', 'wave': 'lh' * 8},
     {'name': 'output1', 'pin': 'D1', 'wave': 'l.h.' * 4},
     {'name': 'output2', 'pin': 'D2', 'wave': 'l...h...' * 2}],
    {},
    ['analysis',
     {'name': 'input0', 'pin': 'D0'},
     {'name': 'input1', 'pin': 'D1'},
     {'name': 'input2', 'pin': 'D2'}]],

    'foot': {'tock': 1},
    'head': {'text': 'loopback_test'}}

waveform = Waveform(loopback_test)
waveform.display()

**Note:** 현재 캡처 된 샘플이 없기 때문에 alnalysis 그룹은 비어 있다.

__Notes on the enhanced WaveJSON specification format__


![](./images/waveform_spec_format3.png)

### Step 2: 패턴 생성기를 실행하고 루프백 신호를 관찰한다.

이 단계는 캡쳐된 trace analyzer samples로 WaveJSON dict를 채울 것이다. DICT는 이제 우리가 추가로 수정할 수있는 결과물로 사용될 수 있다. 이것은 다음 단계에서 표시될 것이다.

In [7]:
pattern_generator = logictools_olay.pattern_generator

pattern_generator.trace(num_analyzer_samples=16)
pattern_generator.setup(loopback_test,
                        stimulus_group_name='stimulus',
                        analysis_group_name='analysis')

pattern_generator.run()
pattern_generator.show_waveform()

### Step 3: waveJSON dict의 출력을  확인한다.

In [9]:
import pprint

output_wavejson = pattern_generator.waveform.waveform_dict
pprint.pprint(output_wavejson)

{'foot': {'tock': 1},
 'head': {'text': 'loopback_test'},
 'signal': [['stimulus',
             {'name': 'output0', 'pin': 'D0', 'wave': 'lhlhlhlhlhlhlhlh'},
             {'name': 'output1', 'pin': 'D1', 'wave': 'l.h.l.h.l.h.l.h.'},
             {'name': 'output2', 'pin': 'D2', 'wave': 'l...h...l...h...'}],
            {},
            ['analysis',
             {'name': 'input0', 'pin': 'D0', 'wave': 'lhlhlhlhlhlhlhlh'},
             {'name': 'input1', 'pin': 'D1', 'wave': 'l.h.l.h.l.h.l.h.'},
             {'name': 'input2', 'pin': 'D2', 'wave': 'l...h...l...h...'}]]}


![](./images/waveform_output_dictionary.png)

### Step 4: State annotation으로 waveJSON dict 출력 확장한다.

In [10]:
state_list = ['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7',
              'S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7']

color_dict = {'white': '2', 'yellow': '3', 'orange': '4', 'blue': '5'}

output_wavejson['signal'].extend([{}, ['Annotation',
                                       {'name': 'state',
                                        'wave': color_dict['yellow'] * 8 +
                                                color_dict['blue'] * 8,
                                        'data': state_list}]])

__Note: __ color_dict는 WaveDrom에 정의 된 색상 코드 맵이다.

In [11]:
draw_wavedrom(output_wavejson)